### Launch Jobs for training

In [ ]:
import os
import ads

from ads.jobs import DataScienceJob
from ads.jobs import DataScienceJobRun
from ads.jobs import ScriptRuntime
from ads.jobs import Job

from ads import set_auth

In [ ]:
# compartment_id = os.environ['NB_SESSION_COMPARTMENT_OCID']
# project_id = os.environ['PROJECT_OCID']

set_auth(auth='resource_principal')

In [ ]:
#
# Here all the definitions
#
LOG_GROUP_ID = "ocid1.loggroup.oc1.eu-frankfurt-1.amaaaaaangencdyazs4l4rzrzsarlej6mqlwlbz6bmnx4adwdlssveam2jaa"
LOG_ID = "ocid1.log.oc1.eu-frankfurt-1.amaaaaaangencdya47httqmxyiew5tkxa6l7gekev2ljpasixuhmp2fa3v5q"

NAMESPACE = "frqap2zhtzbe"
CONDA_BUCKET = "whisper_jobs_env"
# bucket with code to execute
SOURCE_BUCKET = "whisper_jobs"

CUSTOM_ENV_URI = f"oci://{CONDA_BUCKET}@{NAMESPACE}/conda_environments/gpu/whisper_env_/1.0/whisper_env_v1_0"
SOURCE_URI = f"oci://{SOURCE_BUCKET}@{NAMESPACE}/test_italian.tar.gz"

# the first to execute
RUN_ENTRYPOINT = "train_italian.sh"

# SHAPE_NAME = "VM.Standard2.4"
# SHAPE_NAME = "VM.GPU2.1"
SHAPE_NAME = "VM.GPU.A10.2"
# in GB
STORAGE_SIZE = 2000

JOBS_NAME = "job_whisper_italian_010"

In [ ]:
# 1. Specify the Infrastructure requested
# VM Shape, logging
# network is taken from NB session

# you need to provide the OCID for LogGroup and Log
infrastructure = (
    DataScienceJob()
    .with_shape_name(SHAPE_NAME)
    .with_block_storage_size(STORAGE_SIZE)
    .with_log_group_id(LOG_GROUP_ID)
    .with_log_id(LOG_ID)
)

In [ ]:
# specify the runtime and conda and env 
runtime = (
    ScriptRuntime()
    .with_source(SOURCE_URI)
    .with_custom_conda(CUSTOM_ENV_URI)
    .with_environment_variable(JOB_RUN_ENTRYPOINT=RUN_ENTRYPOINT)
)

In [ ]:
# specify the JOB
job = (
    Job(name=JOBS_NAME)
    .with_infrastructure(infrastructure)
    .with_runtime(runtime)
)

In [ ]:
# create the JOB
job.create()

In [ ]:
# get the job run by id
# JOB_RUN_OCID = "ocid1.datasciencejobrun.oc1.eu-frankfurt-1.amaaaaaangencdyais5w2gkzbqbwqeftps6raay65ymhg46hltw3vlokij3q"

# job_run = DataScienceJobRun.from_ocid(JOB_RUN_OCID)

In [ ]:
job_run = job.run()

In [ ]:
job_run.watch()

In [ ]:
%%capture --no-stderr cap
job_run.watch()

with open('output1000_whisper_italian.txt', 'w') as f:
    f.write(cap.stdout)